In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional
import pandas as pd
from torchsummary import summary
from b2aiprep.dataset import VBAIDataset
from b2aiprep.process import Audio, specgram, plot_spectrogram
import IPython.display as Ipd
import os
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

### Audiorecordings(from lectures)

In [49]:
dataset = VBAIDataset('../bids_with_sensitive_recordings')
data = torch.load('../bids_with_sensitive_recordings/sub-0e2df8b3-a93f-4982-a82c-d96a5c64d153/ses-461EA3E8-4477-4F97-B091-D21F4006B2FC/audio/sub-0e2df8b3-a93f-4982-a82c-d96a5c64d153_ses-461EA3E8-4477-4F97-B091-D21F4006B2FC_Audio-Check_rec-Audio-Check-1.pt')
print(data["melfilterbank"].shape)
print(data.keys())

torch.Size([476, 20])
dict_keys(['speaker_embedding', 'specgram', 'melfilterbank', 'mfcc', 'sample_rate', 'opensmile'])


### Demographics

In [4]:
dg = pd.read_csv('../demographics.csv')
dg.head(1)

,Unnamed: 0,record_id,demographics_session_id,demographics_duration,demographics_completed_by___1,demographics_completed_by___2,demographics_completed_by___3,state_province,country,gender_identity,...,household_count,spouse_partner_sig_other,children,parent,grandparent,other_live_with,others_household_specify,transportation_yn,primary_transportation,q_generic_demographics_complete
0,0,8d5dc52b-e8aa-42e7-ae54-8f05c4667d39,B176636C-3330-4AB4-93A9-1E2305506407,173.0,True,False,False,Tennessee,USA,Female gender identity,...,4.0,No,Yes,Yes,No,No,NaN,Yes,Personal vehicle,Complete


In [5]:
X_train = dg[["record_id", "demographics_session_id"]]
Y_income = dg [["household_income_usa", "household_income_ca", "household_count"]]

Train = dg[["record_id", "demographics_session_id", "household_income_usa", "household_income_ca", "household_count"]]
Train.head(3)


,record_id,demographics_session_id,household_income_usa,household_income_ca,household_count
0,8d5dc52b-e8aa-42e7-ae54-8f05c4667d39,B176636C-3330-4AB4-93A9-1E2305506407,"$15,000 to $29,999",NaN,4.0
1,1b07b18b-26f9-405b-a466-29442306a7fe,8F8E68BB-E68C-4EA5-B71A-17D7AAE915C2,NaN,"$150,000 to $199,999",4.0
2,e5db3e0c-6589-4a15-a5e7-8a95e4ed34a5,B94FE4BC-79FF-46A1-86CC-628E2D77874E,NaN,"$50,000 to $99,999",1.0


In [6]:
print("Shape before filtering: ",Train.shape)
pre_train = Train[(pd.notna(Train['household_income_usa']) | pd.notna(Train['household_income_ca'])) &
                                   (~((Train['household_income_usa'] == 'Prefer not to answer') | 
                                      (Train['household_income_ca'] == 'Prefer not to answer')))]
print("Shape after filtering: ", pre_train.shape)


Shape before filtering:  (179, 5)
Shape after filtering:  (115, 5)


In [7]:
#label data: 0 - poverty, 1 - lower, 2 - middle, 3 - upper
pre_train_labeled = pd.DataFrame(pre_train)

for index, row in pre_train.iterrows():
    if pd.notna(pre_train.loc[index, "household_income_usa"]) and pre_train.loc[index, "household_count"] >= 3: # USD; HH >= 3
        income = pre_train.loc[index, "household_income_usa"]
        if income in ['< $15,000', '$15,000 to $29,999']:
            pre_train_labeled.at[index, "SES"] = 0
        elif income in ['$30,000 to $$49,999']:
            pre_train_labeled.at[index, "SES"] = 1
        elif income in ['$50,000 to $99,999', '$100,000 to $149,999', '$150,000 to $199,999']:
            pre_train_labeled.at[index, "SES"] = 2
        elif income in ['$200,000 to $249,999', '>$250,000']:
            pre_train_labeled.at[index, "SES"] = 3
        elif income in ['Prefer not to answer']:
            continue
        else:
            print(income)
            raise ValueError("Wrong value for household_income_usa")
    
    elif pd.notna(pre_train.loc[index, "household_income_usa"]): # USD; HH < 3
        income = pre_train.loc[index, "household_income_usa"]
        if income in ['< $15,000']:
            pre_train_labeled.at[index, "SES"] = 0
        elif income in ['$15,000 to $29,999', '$30,000 to $$49,999']:
            pre_train_labeled.at[index, "SES"] = 1
        elif income in ['$50,000 to $99,999']:
            pre_train_labeled.at[index, "SES"] = 2
        elif income in ['$100,000 to $149,999', '$150,000 to $199,999', '$200,000 to $249,999', '>$250,000']:
            pre_train_labeled.at[index, "SES"] = 3
        elif income in ['Prefer not to answer']:
            continue
        else:
            print(income)
            raise ValueError("Wrong value for household_income_usa")
        
    elif pd.notna(pre_train.loc[index, "household_income_ca"]):  # CA; HH >= 3
        income = pre_train.loc[index, "household_income_ca"]
        if income in ['< $15,000', '$15,000 to $29,999']:
            pre_train_labeled.at[index, "SES"] = 0
        elif income in ['$30,000 to $$49,999']:
            pre_train_labeled.at[index, "SES"] = 1
        elif income in ['$50,000 to $99,999', '$100,000 to $149,999']:
            pre_train_labeled.at[index, "SES"] = 2
        elif income in ['$150,000 to $199,999', '$200,000 to $249,999', '>$250,000']:
            pre_train_labeled.at[index, "SES"] = 3
        elif income in ['Prefer not to answer']:
            continue
        else:
            print(income)
            raise ValueError("Wrong value for household_income_ca")
        
    else:
        print(index)
        
   
one_hot = pd.get_dummies(pre_train_labeled['SES'], prefix='SES').astype(int)

pre_train_labeled = pd.concat([pre_train_labeled, one_hot], axis=1)

# display("Encoded labels: ", pre_train_labeled.head(3))
# print("Shape: ", pre_train_labeled.shape)

Y_label = pre_train_labeled.loc[:, "SES"]
print(Y_label.value_counts())

# Calculate the number of rows that correspond to the first 80%
num_rows = int(len(pre_train_labeled) * 0.8)
pre_train_labeled_first_80 = pre_train_labeled.iloc[:num_rows]

# Extract the "SES" column
Y_label_first_80 = pre_train_labeled_first_80.loc[:, "SES"]

# Get the value counts for the "SES" column
value_counts_first_80 = Y_label_first_80.value_counts()

print("train", value_counts_first_80)

# Calculate the number of rows that correspond to the first 80%
num_rows_2 = int(len(pre_train_labeled) * 0.1)
pre_train_labeled_first_80 = pre_train_labeled.iloc[num_rows:(num_rows+num_rows_2)]

# Extract the "SES" column
Y_label_first_80 = pre_train_labeled_first_80.loc[:, "SES"]

# Get the value counts for the "SES" column
value_counts_first_80 = Y_label_first_80.value_counts()

print("val ", value_counts_first_80)

# Calculate the number of rows that correspond to the first 80%
pre_train_labeled_first_80 = pre_train_labeled.iloc[(num_rows+num_rows_2):]

# Extract the "SES" column
Y_label_first_80 = pre_train_labeled_first_80.loc[:, "SES"]

# Get the value counts for the "SES" column
value_counts_first_80 = Y_label_first_80.value_counts()

print("test", value_counts_first_80)


SES
2.0    37
1.0    35
3.0    29
0.0    14
Name: count, dtype: int64
train SES
2.0    33
1.0    28
3.0    23
0.0     8
Name: count, dtype: int64
val  SES
0.0    5
1.0    3
3.0    2
2.0    1
Name: count, dtype: int64
test SES
3.0    4
1.0    4
2.0    3
0.0    1
Name: count, dtype: int64


### Audiofiles

In [8]:
class RainbowAudioDataset(torch.utils.data.Dataset):
	def __init__(self, data):
		# self.segment_size = segment_size
		self.data = data
		
		# get location for every recording of rainbow passage
		for index, row in data.iterrows():
			subject = "sub-"+row['record_id']
			session = "ses-"+row['demographics_session_id']
			location = str("../bids_with_sensitive_recordings/" + subject + "/" + session + '/audio/'+subject+"_"+session+"_Rainbow-Passage_rec-Rainbow-Passage.wav")
			if os.path.exists(location):
				self.data.at[index, "audio_location"] = location
			else:
				# print all patients without Rainbow passage recording   
				# print(location)
				# data.at[index, "audio_location"] = None
				self.data = self.data.drop(index = index)
		self.data.reset_index(drop=True, inplace=True)
		# self.data = self.data.to_numpy()

	def __len__(self):
		return len(self.data)

	def __getitem__(self, idx):
		audio = Audio.from_file(self.data.loc[idx, "audio_location"])
		audio = audio.to_16khz().signal.squeeze()
		d = (audio.size(0)-48000)//2
		audio = audio[d:d+48000]
		
		ses = self.data.loc[idx, ["SES_0.0", "SES_1.0", "SES_2.0", "SES_3.0"]].to_numpy(dtype=np.float32)
		ses = torch.tensor(ses, dtype=torch.float32)
		
		return {'signal': audio, 'SES': ses}

		# # get middle K seconds if audio is too long, pad with zeros if it is too short
		# if audio.size(0) > self.segment_size*16000:
		# 	d = (audio.size(0)-self.segment_size*16000)//2
		# 	audio = audio[d:d+self.segment_size*16000]
		# else:
		# 	audio = torch.nn.functional.pad(audio, (0,self.segment_size*16000-audio.size(0)), mode='constant', value=0)

	def analyze_length(self):
		total_length = 0
		min_val = 10e9
		max_val = 0
		for idx in range(len(self.data)):
			audio = Audio.from_file(self.data.loc[idx, "audio_location"])
			audio = audio.to_16khz().signal.squeeze()
			length = audio.size(0)  # Number of samples in the audio
			min_val = min_val if min_val < length else length
			max_val = max_val if max_val > length else length
			total_length += length
		average_length = total_length / len(self.data)
		return average_length, min_val, max_val

In [9]:
display(pre_train_labeled)
dataset = RainbowAudioDataset(pre_train_labeled)
avg, min_val, max_val = dataset.analyze_length()
print(avg, min_val, max_val)

,record_id,demographics_session_id,household_income_usa,household_income_ca,household_count,SES,SES_0.0,SES_1.0,SES_2.0,SES_3.0
0,8d5dc52b-e8aa-42e7-ae54-8f05c4667d39,B176636C-3330-4AB4-93A9-1E2305506407,"$15,000 to $29,999",NaN,4.0,0.0,1,0,0,0
1,1b07b18b-26f9-405b-a466-29442306a7fe,8F8E68BB-E68C-4EA5-B71A-17D7AAE915C2,NaN,"$150,000 to $199,999",4.0,3.0,0,0,0,1
2,e5db3e0c-6589-4a15-a5e7-8a95e4ed34a5,B94FE4BC-79FF-46A1-86CC-628E2D77874E,NaN,"$50,000 to $99,999",1.0,2.0,0,0,1,0
3,943a8bbc-bba0-4853-825c-10cae1b26ddd,35002E1F-A963-4E9C-8F2A-738839D83C4D,"$15,000 to $29,999",NaN,1.0,1.0,0,1,0,0
4,d5aeba9f-c910-4b65-81e8-0e9ad15097e7,8B341FFA-28B5-46B8-97D1-2B68F5882185,"$50,000 to $99,999",NaN,2.0,2.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
170,2de86669-cc1f-45ef-9d31-a7eabd68f247,07E782FE-DF6F-4CFF-BA9A-1A394845E0B3,"$150,000 to $199,999",NaN,2.0,3.0,0,0,0,1
171,92425dec-8998-43bf-adfc-00414c66b9a2,BD5B8396-2ACB-4FA9-97CE-4498C46023D6,">$250,000",NaN,2.0,3.0,0,0,0,1
172,28cc3eda-66eb-4731-b5fa-e59f2cbaf801,744EA686-2B23-4770-980B-96D4D2962D85,NaN,"$50,000 to $99,999",2.0,2.0,0,0,1,0
176,dbd7641f-f3db-4d08-8f63-2e0e7bbb8f3d,7DCE811B-CDAE-4DAB-B2B6-EDE0BF9534D4,"$150,000 to $199,999",NaN,2.0,3.0,0,0,0,1


464351.10714285716 282947 808798


In [31]:
X = pre_train_labeled[['record_id', 'demographics_session_id']]
y = pre_train_labeled[['SES', 'SES_0.0', 'SES_1.0', 'SES_2.0', 'SES_3.0']]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)

print(y_train.value_counts())
print(y_val.value_counts())
print(y_test.value_counts())

train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

N = len(pre_train_labeled)

train_dataset = RainbowAudioDataset(pre_train_labeled[:int(0.8*N)])
val_dataset = RainbowAudioDataset(pre_train_labeled[int(0.8*N):int(0.9*N)])
test_dataset = RainbowAudioDataset(pre_train_labeled[int(0.9*N):])

# train_dataset = RainbowAudioDataset(train_df)
# val_dataset = RainbowAudioDataset(val_df)
# test_dataset = RainbowAudioDataset(test_df)

# print(train_dataset[1])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

for batch in train_dataloader:
    print(batch['signal'].shape)
    print(batch['SES'].shape)
    break


SES  SES_0.0  SES_1.0  SES_2.0  SES_3.0
2.0  0        0        1        0          30
1.0  0        1        0        0          28
3.0  0        0        0        1          23
0.0  1        0        0        0          11
Name: count, dtype: int64
SES  SES_0.0  SES_1.0  SES_2.0  SES_3.0
1.0  0        1        0        0          3
2.0  0        0        1        0          3
3.0  0        0        0        1          3
0.0  1        0        0        0          2
Name: count, dtype: int64
SES  SES_0.0  SES_1.0  SES_2.0  SES_3.0
1.0  0        1        0        0          4
2.0  0        0        1        0          4
3.0  0        0        0        1          3
0.0  1        0        0        0          1
Name: count, dtype: int64
torch.Size([16, 48000])
torch.Size([16, 4])


### CNN Model

In [51]:
class CNN_1D(torch.nn.Module):
    def __init__(self, num_classes):
        super(CNN_1D, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=11, stride=32, padding=0)
        self.mp = nn.MaxPool1d(kernel_size=3, stride=3)
        self.dropout = nn.Dropout(0.5)
  
        # Calculate the input size to the linear layer
        self.fc_input_size = self._calculate_fc_input_size()
  
        self.fc = torch.nn.Linear(self.fc_input_size, num_classes)

    def forward(self, x):
        x = self.mp(torch.nn.functional.relu(self.conv1(x)))
        x = x.reshape(x.size(0), -1)
        x = self.dropout(x)  # Apply dropout before the linear layer
        x = self.fc(x)
        return x

    def _calculate_fc_input_size(self):
        # Example calculation assuming input size (1, 48000)
        x = torch.randn(1, 1, 48000)
        x = self.mp(nn.functional.relu(self.conv1(x)))
        return x.view(x.size(0), -1).size(1)



In [52]:
# cnn = CNN_1D(4)
cnn = CNN_1D(num_classes=4)
_ = summary(cnn, (1, 48000))
# for batch in train_dataloader:
#     out = cnn(batch['signal'].unsqueeze(1)).squeeze(1)[0]
#     out = torch.tensor([0.1, 0.2, 1, 10])
#     print(out)
#     print(torch.nn.functional.softmax(out))
#     print()
#     break

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 64, 1500]             768
         MaxPool1d-2              [-1, 64, 500]               0
           Dropout-3                [-1, 32000]               0
            Linear-4                    [-1, 4]         128,004
Total params: 128,772
Trainable params: 128,772
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.18
Forward/backward pass size (MB): 1.22
Params size (MB): 0.49
Estimated Total Size (MB): 1.90
----------------------------------------------------------------


In [53]:
def eval(model, dataloader, verbose = False):
	model.eval()
	acc = 0
	total = 0
	for batch in dataloader:
		with torch.no_grad():
			outputs = torch.nn.functional.softmax(model(batch['signal'].unsqueeze(1)).squeeze(1))
			if verbose:
				print(outputs)
		for i in range(len(batch['signal'])):
			_, predicted = torch.max(outputs, 1)
			_, ground_truth = torch.max(batch['SES'], 1)
			acc += (predicted == ground_truth).sum().item()
			total += ground_truth.size(0)
			# _, predicted = torch.max(outputs, 1)
			# _, ground_truth = torch.max(batch['SES'], 1)
			# acc += (predicted == ground_truth).sum().item()
	if verbose:
		return acc/total, outputs
	return acc/total	

optimizer = torch.optim.Adam(cnn.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
criterion = nn.CrossEntropyLoss()

num_epochs = 20
best_val_acc = 0

for epoch in range(num_epochs):
    cnn.train()
    closs = []
    for batch in train_dataloader:        
        optimizer.zero_grad()
        outputs = cnn(batch['signal'].unsqueeze(1))
        loss = criterion(outputs, batch['SES'].argmax(dim=1))
        closs += [loss.item()] * len(batch['signal'])
        loss.backward()
        optimizer.step()
    scheduler.step()

    val_acc = eval(cnn, val_dataloader)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(cnn.state_dict(), 'mymodel.pth')
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {sum(closs)/len(closs)}, Val Acc: {val_acc}')


Epoch 1/20, Loss: 11.291212971290845, Val Acc: 0.09090909090909091
Epoch 2/20, Loss: 7.2851874426509555, Val Acc: 0.18181818181818182
Epoch 3/20, Loss: 3.54530735230178, Val Acc: 0.36363636363636365
Epoch 4/20, Loss: 1.684962461503704, Val Acc: 0.2727272727272727
Epoch 5/20, Loss: 0.9966382169991397, Val Acc: 0.2727272727272727
Epoch 6/20, Loss: 0.7092560924840777, Val Acc: 0.18181818181818182
Epoch 7/20, Loss: 0.63192523396417, Val Acc: 0.18181818181818182
Epoch 8/20, Loss: 0.4663518908318509, Val Acc: 0.18181818181818182
Epoch 9/20, Loss: 0.37390894802768576, Val Acc: 0.18181818181818182
Epoch 10/20, Loss: 0.3117556138319916, Val Acc: 0.18181818181818182
Epoch 11/20, Loss: 0.25304443916577973, Val Acc: 0.18181818181818182
Epoch 12/20, Loss: 0.23693065462487467, Val Acc: 0.18181818181818182
Epoch 13/20, Loss: 0.2224369916353333, Val Acc: 0.18181818181818182
Epoch 14/20, Loss: 0.21841906563619548, Val Acc: 0.18181818181818182
Epoch 15/20, Loss: 0.20201160197847345, Val Acc: 0.181818181

In [54]:
cnn.load_state_dict(torch.load('./mymodel.pth'))
test_acc, outputs = eval(cnn, test_dataloader, verbose=True)
print(outputs.argmax(dim=1))
print("      ", y_test.loc[:, "SES"].to_numpy())
print('TestACC:{:.4f}'.format(test_acc))

tensor([[4.1345e-01, 1.1921e-03, 4.7476e-01, 1.1060e-01],
        [5.1693e-01, 9.9078e-04, 4.2057e-01, 6.1507e-02],
        [4.1406e-01, 1.8970e-04, 4.1604e-01, 1.6971e-01],
        [3.1798e-01, 9.1318e-04, 6.0316e-01, 7.7950e-02],
        [3.3482e-01, 9.1574e-04, 5.9679e-01, 6.7476e-02],
        [3.4803e-01, 8.2322e-04, 5.0423e-01, 1.4692e-01],
        [3.3179e-01, 9.4600e-04, 5.9848e-01, 6.8787e-02],
        [3.4281e-01, 8.9685e-04, 5.8859e-01, 6.7711e-02],
        [3.4344e-01, 9.3260e-04, 5.8647e-01, 6.9160e-02],
        [3.2558e-01, 8.2354e-04, 6.0175e-01, 7.1856e-02],
        [5.6935e-01, 9.8688e-04, 2.6370e-01, 1.6597e-01],
        [3.3380e-01, 9.4796e-04, 5.9742e-01, 6.7834e-02]])
tensor([2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2])
       [1. 2. 3. 1. 2. 1. 2. 3. 2. 0. 3. 1.]
TestACC:0.2500


In [15]:
# Just for plotting

test_loss, test_accuracy = cnn.evaluate(test_features, test_labels)
print(f'Test Accuracy: {test_accuracy}')


plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

test_predictions = (cnn.predict(test_features) > 0.5).astype("int32")

# Confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['Not Race_5', 'Race_5'])
disp.plot(cmap=plt.cm.Blues)
plt.show()


AttributeError: 'CNN_1D' object has no attribute 'evaluate'